# 세이지메이커 점프스타트에서 Llama 2 모델의 연속 사전 훈련

In [ ]:
%pip install -U datasets==2.15.0

In [ ]:
%pip install -U --force-reinstall \
             langchain==0.0.324 \
             typing_extensions==4.7.1 \
             pypdf==3.16.4

## 사전 훈련된 모델 배포하기

---

먼저 Llama 2 모델을 세이지메이커 엔드포인트로 배포합니다. 13B이나 70B 모델을 훈련하거나 배포하려면 모델 ID를 각각 "meta-textgeneration-llama-2-13b" 및 "meta-textgeneration-llama-2-70b"로 변경하세요.

---

In [ ]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel

pretrained_model = JumpStartModel(model_id=model_id, model_version=model_version)
pretrained_predictor = pretrained_model.deploy()

## 엔드포인트 호출하기

---

다음으로 몇 가지 샘플 쿼리를 사용하여 엔드포인트를 호출합니다. 이후에 이 노트북에서는 해당 모델을 사용자 정의 데이터 세트로 미세 조정하고, 미세 조정된 모델을 사용하여 추론을 수행합니다. 또한 사전 훈련된 모델과 미세 조정된 모델을 통해 얻은 결과를 비교합니다.

---

In [ ]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response[0]['generation']}")
    print("\n==================================\n")

In [ ]:
# payload = {
#     "inputsWhat is the size of the Amazon consumer business in 2022?e is",
#     "parameters": {
#         "max_new_token_p": 0.9,
#         "temperature": 0.6,
#         "return_full_text": False,
#     },
# }
# try:
#     response = pretrained_predictor.predict(payload, custom_attributes="accept_eula=true")
#     print_response(payload, response)
# except Exception as e:
#     print(e)

### 연속 사전 훈련을 위한 데이터 세트 형식화

#### 도메인 적응 미세 조정

텍스트 생성 모델은 특정 도메인 데이터 세트에서도 미세 조정될 수 있습니다. 특정 도메인 데이터 세트로 미세 조정된 후 모델은 **적은샷 프롬프트(few-shot prompting)** 를 사용하여 해당 도메인에서 도메인 특화 텍스트를 생성하고 다양한 NLP 작업을 해결할 수 있을 것으로 기대됩니다.

아래는 모델에 입력할 훈련 데이터 형식에 대한 지침입니다:

- **입력:** 훈련 및 선택 사항인 검증 디렉토리. 각 디렉토리에는 CSV/JSON/TXT 파일이 포함되어 있습니다.
  - CSV/JSON 파일의 경우 'text'라는 열에서 훈련 또는 검증 데이터를 사용하거나 'text' 열이 없으면 첫 번째 열을 사용합니다.
  - 훈련 및 검증(제공된 경우) 디렉토리의 파일 수는 각각 하나여야 합니다. 
- **출력:** 추론을 위해 배포할 수 있는 훈련된 모델.

In [ ]:
s3_prefix = f"s3://jumpstart-cache-prod-{boto3.Session().region_name}/training-datasets/sec_amazon"
s3_location = f"s3://jumpstart-cache-prod-{boto3.Session().region_name}/training-datasets/sec_amazon/AMZN_2021_2022_train_js.txt"

In [ ]:
!aws s3 cp $s3_location ./

In [ ]:
!head AMZN_2021_2022_train_js.txt

In [ ]:
import boto3
from sagemaker.jumpstart.estimator import JumpStartEstimator

model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

estimator = JumpStartEstimator(model_id=model_id, 
                               model_version=model_version, 
                               environment={"accept_eula": "true"},
                               instance_type = "ml.g5.24xlarge")

estimator.set_hyperparameters(instruction_tuned="False", epoch="5")

estimator.fit({"training": s3_prefix})

### 연속 사전 훈련된 모델 배포하기

In [ ]:
cpt_predictor = estimator.deploy()

In [ ]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response[0]['generation']}")
    print("\n==================================\n")

In [ ]:
payload = {
    "inputs": "What is the size of the Amazon consumer business in 2022?",
    "parameters": {"max_new_tokens": 100},
}

try:
    response = cpt_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

In [ ]:
# # 리소스 삭제
# pretrained_predictor.delete_model()
# pretrained_predictor.delete_endpoint()
# cpt_predictor.delete_model()
# cpt_predictor.delete_endpoint()